--- Day 3: Rucksack Reorganization ---

One Elf has the important job of loading all of the [rucksacks](https://en.wikipedia.org/wiki/Rucksack) with supplies for the jungle journey. Unfortunately, that Elf didn't quite follow the packing instructions, and so a few items now need to be rearranged.

Each rucksack has two large **compartments**. All items of a given type are meant to go into exactly one of the two compartments. The Elf that did the packing failed to follow this rule for exactly one item type per rucksack.

The Elves have made a list of all of the items currently in each rucksack (your puzzle input), but they need your help finding the errors. Every item type is identified by a single lowercase or uppercase letter (that is, a and A refer to different types of items).

The list of items for each rucksack is given as characters all on a single line. A given rucksack always has the same number of items in each of its two compartments, so the first half of the characters represent items in the first compartment, while the second half of the characters represent items in the second compartment.

For example, suppose you have the following list of contents from six rucksacks:
```
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
```
- The first rucksack contains the items `vJrwpWtwJgWrhcsFMMfFFhFp`, which means its first compartment contains the items `vJrwpWtwJgWr`, while the second compartment contains the items `hcsFMMfFFhFp`. The only item type that appears in both compartments is lowercase `p`.
- The second rucksack's compartments contain `jqHRNqRjqzjGDLGL` and `rsFMfFZSrLrFZsSL`. The only item type that appears in both compartments is uppercase `L`.
- The third rucksack's compartments contain `PmmdzqPrV` and `vPwwTWBwg`; the only common item type is uppercase `P`.
- The fourth rucksack's compartments only share item type `v`.
- The fifth rucksack's compartments only share item type `t`.
- The sixth rucksack's compartments only share item type `s`.
To help prioritize item rearrangement, every item type can be converted to a **priority**:

-Lowercase item types `a` through `z` have priorities `1` through `26`.
-Uppercase item types `A` through `Z` have priorities `27` through `52`.
In the above example, the priority of the item type that appears in both compartments of each rucksack is `16` (`p`), `38` (`L`), `42` (`P`), `22` (`v`), `20` (`t`), and `19` (`s`); the sum of these is `157`.

Find the item type that appears in both compartments of each rucksack. **What is the sum of the priorities of those item types?**

In [2]:
_a = ord('a')
_z = ord('z')
_A = ord('A')
_Z = ord('Z')
def item_to_priority(item):
    priority = ord(item)
    
    if _a <= priority <= _z:
        return priority - _a + 1
    if _A <= priority <= _Z:
        return priority - _A + 27
    
    raise RuntimeError(f"Could not find priority for {item}")
    
def find_single_misplaced_item(rucksack):
    left = rucksack[:len(rucksack)//2]
    right = rucksack[len(rucksack)//2:]
    
    for item in left:
        if item in right:
            return item
        
def get_priorites_of_rucksacks(rucksacks):
    return sum([item_to_priority(find_single_misplaced_item(x)) for x in rucksacks])

def test():
    def _text_example_1():
        example_input = [
            "vJrwpWtwJgWrhcsFMMfFFhFp",
            "jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL",
            "PmmdzqPrVvPwwTWBwg",
            "wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn",
            "ttgJtRGJQctTZtZT",
            "CrZsJsPPZsGzwwsLwLmpwMDw",
        ]
        
        expected = 157
        actual = get_priorites_of_rucksacks(example_input)
        
        assert actual == expected
    _text_example_1()
    
    def _text_example_2():
        pass
    _text_example_2()
test()

In [4]:
with open("day_3_input.txt") as _file:
    lines = _file.readlines()

In [6]:
def solution1():
    print(get_priorites_of_rucksacks(lines))
solution1()

8233
